In [1]:
from subprocess import Popen, PIPE
file_exe="../../../Mzinga.LinuxX64/MzingaEngine"
player = Popen(file_exe, stdin=PIPE, stdout=PIPE, stderr=PIPE, encoding="UTF8", bufsize=1)

player.stdout.readline()
player.stdout.readline()
player.stdout.readline()
from game_rule import DLLGameRule 
GR=DLLGameRule()
import time
import random
import ctypes
import math
import time



In [2]:

interactionProgram=""
def initNewGame(player,gamemode=0):
    global interactionProgram
    interactionProgram=""
    match gamemode:
        case 0:
            b="Base"
        case 1:
            b="Base+M"
        case 2:
            b="Base+L"
        case 3:
            b="Base+P"
        case 4:
            b="Base+ML"
        case 5:
            b="Base+MP"
        case 6:
            b="Base+LP"
        case 7:
            b="Base+MLP"
            
        
    player.stdin.write("newgame "+b+"\n")
    s="newgame "+b+"\n"
    #interactionProgram+=s
    s=player.stdout.readline()
    #interactionProgram+=" < "+s+"\n"
    s=player.stdout.readline()
    #interactionProgram+=" < "+s+"\n"
    
def getValidMoves(player):
    global interactionProgram
    player.stdin.write("validmoves\n")
    #interactionProgram+=" > validmoves\n"
    read=player.stdout.readline()
    s=read[:-1].split(";")  
    #interactionProgram+=" < "+read+"\n"
    s=player.stdout.readline()
    #interactionProgram+=" < "+s+"\n"
    return s

def playMove(p):
    global interactionProgram
    global risAct
    player.stdin.write("play "+p+"\n")
    #interactionProgram+=" > play "+p+"\n"
    risAct=player.stdout.readline()
    #interactionProgram+=" < "+risAct
    checkOk=player.stdout.readline()
    #interactionProgram+=" < "+checkOk
    if checkOk=="ok\n":
        return 0
    return checkOk
def playBoth(p,state,act):
    playMove(act)
    GR.next_state(state,GR.stringToAction(state,act))
    
    

In [3]:
initNewGame(player,1)
state = GR.init_state(1)

#GR.next_state(state,9)

#[GR.boardToCVect_low(state)[i] for i in range(284)]


GT:1


In [8]:
evN=0
v1=[]
v2=[]
v2r=[]
moves="";
allVe=[]

mode2=True;
def test():
    global v1
    global mode2
    global v2
    global v2r
    global allVe
    global geConf
    state = None
    for Q in range(10):
        actStrTE=[]
        actStrC=[]
        actStrE=[]
        actNumE=[]
        
        actNumC=[]
        geConf=""
        state = GR.init_state(7)
        initNewGame(player,7)

        iM = 0
        totMov = 0
        while iM <8000:
            global evN

            actTMPC = GR.getActions(state)
            actNumC=[actTMPC[i] for i in range(1,actTMPC[0]+1)]
            actStrC=[GR.actionToString(x,state) for x in actNumC]
            actNumC.sort()
            actStrC.sort()
            
            if not mode2:
                actStrTE = getValidMoves(player)
                actNumE=[GR.stringToAction(state,mov) for mov in actStrTE]
                
                actStrE=[GR.actionToString(x,state) for x in actNumE]
                actNumE.sort()
                actStrE.sort()
                
            
            lE=len(actNumE)
            lC=len(actNumC)
            if lE!=lC and not mode2:
                print("ERROR: Different lenght")
                v1=[GR.actionToString(x,state) for x in actNumE]
                v2=[GR.actionToString(x,state) for x in actNumC]
                v2r=actNumC
                allVe=(actStrTE,actTMPC,actNumE,actNumC,actStrE,actStrC)
                print(geConf)
                return
            else:
                for i in range(lE):
                    if(actStrC[i]!=actStrE[i]) and not mode2:
                        print("ERROR : not same actions")
                        v1=[GR.actionToString(x,state) for x in actNumE]
                        v2=[GR.actionToString(x,state) for x in actNumC]
                        allVe=(actTMPC,actStrTE,actNumE,actNumC,actStrE,actStrC)
                        v2r=actNumC
                        print(geConf)
                        return
            # scelta casuale per il secondo giocatore
            
            m = random.randint(1, lC)
            geConf+=(f"{iM+1}. {actStrC[m-1]}\n")
            
            #play move
            act=actStrC[m-1]
            ris=playMove(act)
            if(ris!=0):
                print("ERROR : not same actions, message status:_",ris,"_", len(ris))
                v1=[GR.actionToString(x,state) for x in actNumE]
                v2=[GR.actionToString(x,state) for x in actNumC]
                allVe=(actTMPC,actStrTE,actNumE,actNumC,actStrE,actStrC)
                v2r=actNumC
                print(geConf)
                return
            GR.next_state(state,GR.stringToAction(state,act))
        
            #GR.next_state(state,GR.stringToAction(state,actStrC[m-1]))
    
            totMov += 1
            iM += 1
            
            status = GR.checkStatus(state)
            if status != 1:
                print("totMoves:", iM)
                iM=100000
                break
        print("Q:",Q)
        GR.delBoard(state)
test()

GT:7
totMoves: 110
Q: 0
GT:7
totMoves: 386
Q: 1
GT:7
totMoves: 2381
Q: 2
GT:7
totMoves: 147
Q: 3
GT:7
totMoves: 72
Q: 4
GT:7
totMoves: 184
Q: 5
GT:7
totMoves: 1866
Q: 6
GT:7
Q: 7
GT:7
totMoves: 72
Q: 8
GT:7
Q: 9


In [7]:
l1=len(v1)
l2=len(v2)
l1,l2


(0, 0)

In [10]:
GR.getMask_low=GR.dll._Z7getMaskP6EBoard
GR.getMask_low.argtypes = [ctypes.c_void_p]
GR.getMask_low.restype = ctypes.POINTER(ctypes.c_int8)

In [11]:
state

33263120

In [17]:
ptr = GR.getMask_low(state)

In [18]:
import numpy as np

In [37]:
arr = np.ctypeslib.as_array(ptr, shape=(1575,))

In [38]:
v1.sort()
v2.sort()

In [39]:
for i in range(max(l1,l2)):
    if((v1[i] if i<l1 else "       ")!=(v2[i] if i<l2 else 0)):
        print(v1[i] if i<l1 else "       ","\t",v2[i] if i<l2 else 0)

In [40]:
ris=arr.copy().astype(bool)


In [46]:
sum(1 if ris[i]==True else 0 for i in range(1575))


5

In [47]:
ra=GR.getActions(state)

In [48]:
ra[0]

5

In [ ]:
totMov = 0
t_start = time.time()

for rt in range(5):
    initNewGame(player)    
    i = 0

    while i<200000:

        # .EBoard_getNextsActions(board, ris)
        actStrE = getValidMoves(player)

        num_actions = len(actStrE)
        m = random.randint(1, num_actions)
        if num_actions<=1:
            print(i)
            break
        playMove(actStrE[m-1])
        
        totMov += 1
        
        i += 1

    
t_end = time.time()
print("Tempo totale mossa", (t_end - t_start) /totMov, "s")
print(totMov)


In [ ]:
(9227321486950268931>>56)&31


In [ ]:
import numpy as np

In [ ]:
a=np.unpackbits(np.ctypeslib.asarray(

In [ ]:
state = GR.init_state(1)

GR.getActions(state)


In [ ]:
GM=GR.dll._Z7getMaskP6EBoard
GM.argtypes = [ctypes.c_void_p]
GM.restype=np.ctypeslib.ctypes.pointer(np.ctypeslib.ctypes.c_uint8)
#ctypes.POINTER(ctypes.c_uint8)

In [ ]:
ris1=GM(state)

In [ ]:
ris1


In [ ]:
s=0
for i in range(1575):
    s+=ris1[i]
print(s)

In [ ]:
np.ctypeslib.as_array(ris1,np.zeros(1575))

In [ ]:
ris1

In [ ]:
z=np.zeros(30)

In [ ]:
z.ctypes